# **Виртуальная стажировка Shift + Enter по направлению Data Science**

## Задание 1. Тренировка классификатора

In [1]:
# загрузка датасета
from datasets import load_dataset, concatenate_datasets

df = load_dataset("AmazonScience/massive", "en-US")

Found cached dataset massive (/home/sergey/.cache/huggingface/datasets/AmazonScience___massive/en-US/1.0.0/71d360eb7d7a18565ff8c10609cebf714fce3cc390e173ba5b02ffd48543cdc1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
df

DatasetDict({
    train: Dataset({
        features: ['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 11514
    })
    validation: Dataset({
        features: ['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 2033
    })
    test: Dataset({
        features: ['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 2974
    })
})

In [3]:
# первая строчка из трейн датасета
df["train"][0]

{'id': '1',
 'locale': 'en-US',
 'partition': 'train',
 'scenario': 16,
 'intent': 48,
 'utt': 'wake me up at nine am on friday',
 'annot_utt': 'wake me up at [time : nine am] on [date : friday]',
 'worker_id': '1',
 'slot_method': {'slot': [], 'method': []},
 'judgments': {'worker_id': [],
  'intent_score': [],
  'slots_score': [],
  'grammar_score': [],
  'spelling_score': [],
  'language_identification': []}}

Для классификации интента нам нужны лишь тексты запросов. Выберем только колонки `intent` и `utt`

In [4]:
df = df.select_columns(column_names=["intent", "utt"]).rename_columns({"intent":"label", "utt":"text"})
df

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 11514
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 2033
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 2974
    })
})

Взглянем подробнее на фичу `intent` из `train` сплита нашего датасета

In [5]:
# первая строка из трейна
df["train"][0]

{'label': 48, 'text': 'wake me up at nine am on friday'}

In [6]:
# уникальные значения интента
import numpy as np

np.unique(df["train"]["label"])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59])

In [7]:
# другое представление intent
df["train"].features["label"]

ClassLabel(names=['datetime_query', 'iot_hue_lightchange', 'transport_ticket', 'takeaway_query', 'qa_stock', 'general_greet', 'recommendation_events', 'music_dislikeness', 'iot_wemo_off', 'cooking_recipe', 'qa_currency', 'transport_traffic', 'general_quirky', 'weather_query', 'audio_volume_up', 'email_addcontact', 'takeaway_order', 'email_querycontact', 'iot_hue_lightup', 'recommendation_locations', 'play_audiobook', 'lists_createoradd', 'news_query', 'alarm_query', 'iot_wemo_on', 'general_joke', 'qa_definition', 'social_query', 'music_settings', 'audio_volume_other', 'calendar_remove', 'iot_hue_lightdim', 'calendar_query', 'email_sendemail', 'iot_cleaning', 'audio_volume_down', 'play_radio', 'cooking_query', 'datetime_convert', 'qa_maths', 'iot_hue_lightoff', 'iot_hue_lighton', 'transport_query', 'music_likeness', 'email_query', 'play_music', 'audio_volume_mute', 'social_post', 'alarm_set', 'qa_factoid', 'calendar_set', 'play_game', 'alarm_remove', 'lists_remove', 'transport_taxi', 'r

In [8]:
# извлечем имена из структуры выше
df["train"].features["label"].names

['datetime_query',
 'iot_hue_lightchange',
 'transport_ticket',
 'takeaway_query',
 'qa_stock',
 'general_greet',
 'recommendation_events',
 'music_dislikeness',
 'iot_wemo_off',
 'cooking_recipe',
 'qa_currency',
 'transport_traffic',
 'general_quirky',
 'weather_query',
 'audio_volume_up',
 'email_addcontact',
 'takeaway_order',
 'email_querycontact',
 'iot_hue_lightup',
 'recommendation_locations',
 'play_audiobook',
 'lists_createoradd',
 'news_query',
 'alarm_query',
 'iot_wemo_on',
 'general_joke',
 'qa_definition',
 'social_query',
 'music_settings',
 'audio_volume_other',
 'calendar_remove',
 'iot_hue_lightdim',
 'calendar_query',
 'email_sendemail',
 'iot_cleaning',
 'audio_volume_down',
 'play_radio',
 'cooking_query',
 'datetime_convert',
 'qa_maths',
 'iot_hue_lightoff',
 'iot_hue_lighton',
 'transport_query',
 'music_likeness',
 'email_query',
 'play_music',
 'audio_volume_mute',
 'social_post',
 'alarm_set',
 'qa_factoid',
 'calendar_set',
 'play_game',
 'alarm_remove',
 

Стало понятно, что `intent` - это индексы из списка категорий.

Теперь надо токенизировать датасет, чтобы модель поняла естественный язык

In [9]:
# инициализируем токенайзер с BERT (distilled)
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [10]:
# токенезируем все, что имеем
# с обрезанием текста под модель и динамической подачей

def utt_tokenization(data):
    return tokenizer(data["text"], 
                     truncation=True, 
                     padding=True)


tokenized_df = df.map(utt_tokenization, batched=True)
tokenized_df

Loading cached processed dataset at /home/sergey/.cache/huggingface/datasets/AmazonScience___massive/en-US/1.0.0/71d360eb7d7a18565ff8c10609cebf714fce3cc390e173ba5b02ffd48543cdc1/cache-9292b9ec0b3887b6.arrow


Loading cached processed dataset at /home/sergey/.cache/huggingface/datasets/AmazonScience___massive/en-US/1.0.0/71d360eb7d7a18565ff8c10609cebf714fce3cc390e173ba5b02ffd48543cdc1/cache-f454366645a0ab6b.arrow
Loading cached processed dataset at /home/sergey/.cache/huggingface/datasets/AmazonScience___massive/en-US/1.0.0/71d360eb7d7a18565ff8c10609cebf714fce3cc390e173ba5b02ffd48543cdc1/cache-89d4ffaa8d3fb300.arrow


DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 11514
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 2033
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 2974
    })
})

Далее загрузим методы оценки

Используемые метрики:
- accuracy - точность модели в целом
- precision (macro) - точность в предсказании правильных значений
- recall (macro) - доля нахождения правильных значений
- roc_auc (macro) - среднее значение площадей под кривыми, показывающими зависимость между чувствительностью и специфичностью для каждого класса.

Чем ближе эти метрики к 1, тем лучше

In [11]:
# загрузка метрик
import evaluate

accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision", "multiclass")
recall = evaluate.load("recall", "multiclass")
roc_auc = evaluate.load("roc_auc", "multiclass")

In [12]:
# словари для лейблов и их индексов в обе стороны, чтобы модель разобралась, что предсказывать
# также сохраним индексы лейблов для интентов, они понадобятся позже

label2id = {v: i for i, v in enumerate(df["train"].features["label"].names)}
id2label = {i: v for i, v in enumerate(df["train"].features["label"].names)}
whole_labels = np.unique(df["train"]["label"])

In [13]:
# функция для вычисления метрик
from torch import nn
import torch


def compute_metrics(model_pred):
    logits, labels = model_pred
    
    # преобразуем выхлоп модели в предсказания, найдя индексы максимальных элементов
    # по столбцам
    predictions = np.argmax(logits, axis=-1)

    # при помощи функции активации softmax преобразуем выхлоп модели в вероятности
    # softmax: e^(x_i)/sum(e^(x_0),...,e^(x_n)), если мне память не изменяет
    predictions_proba = nn.functional.softmax(torch.from_numpy(logits), dim=-1)

    scores = {
        "accuracy":accuracy.compute(predictions=predictions, 
                                    references=labels),
        "precision":precision.compute(predictions=predictions, 
                                      references=labels, 
                                      average="macro"),
        "recall":recall.compute(predictions=predictions, 
                                references=labels, 
                                average="macro"),

        # так как при глубоком обучении могут отброситься какие-то лейблы,
        # то метод оценивания "один против всех" не сработает (будет жаловаться на несовпадение
        # размерности матрицы вероятностей и референсов), поэтому применяем метод "один против 
        # одного" и явно задаем список лейблов, который мы сохранили в ячейке выше
        "roc_auc":roc_auc.compute(prediction_scores=predictions_proba, 
                                  references=labels, 
                                  labels=whole_labels,
                                  average="macro", 
                                  multi_class="ovo")
    }
    return scores

Теперь начинаем подготовку к обучению

In [14]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# инициализация модели
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(df["train"].features["label"].names),
    id2label=id2label,
    label2id=label2id,
)

# аргументы для обучения
training_args = TrainingArguments(output_dir="intent-class-model")

# объект учителя
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_df["train"],
    eval_dataset=concatenate_datasets([tokenized_df["validation"], tokenized_df["test"]]),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Начинаем обучение

In [15]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/home/sergey/anaconda3/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 11514
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4320
  Number of trainable parameters = 66999612


  0%|          | 0/4320 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Saving model checkpoint to intent-class-model/checkpoint-500
Configuration saved in intent-class-model/checkpoint-500/config.json


{'loss': 2.1551, 'learning_rate': 4.4212962962962966e-05, 'epoch': 0.35}


Model weights saved in intent-class-model/checkpoint-500/pytorch_model.bin
tokenizer config file saved in intent-class-model/checkpoint-500/tokenizer_config.json
Special tokens file saved in intent-class-model/checkpoint-500/special_tokens_map.json
Saving model checkpoint to intent-class-model/checkpoint-1000
Configuration saved in intent-class-model/checkpoint-1000/config.json


...

Model weights saved in intent-class-model/checkpoint-3500/pytorch_model.bin
tokenizer config file saved in intent-class-model/checkpoint-3500/tokenizer_config.json
Special tokens file saved in intent-class-model/checkpoint-3500/special_tokens_map.json
Saving model checkpoint to intent-class-model/checkpoint-4000
Configuration saved in intent-class-model/checkpoint-4000/config.json


{'loss': 0.1723, 'learning_rate': 3.7037037037037037e-06, 'epoch': 2.78}


Model weights saved in intent-class-model/checkpoint-4000/pytorch_model.bin
tokenizer config file saved in intent-class-model/checkpoint-4000/tokenizer_config.json
Special tokens file saved in intent-class-model/checkpoint-4000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 2928.2046, 'train_samples_per_second': 11.796, 'train_steps_per_second': 1.475, 'train_loss': 0.5841062589927956, 'epoch': 3.0}


TrainOutput(global_step=4320, training_loss=0.5841062589927956, metrics={'train_runtime': 2928.2046, 'train_samples_per_second': 11.796, 'train_steps_per_second': 1.475, 'train_loss': 0.5841062589927956, 'epoch': 3.0})

In [16]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5007
  Batch size = 8


  0%|          | 0/626 [00:00<?, ?it/s]

/home/sergey/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.5203498601913452,
 'eval_accuracy': {'accuracy': 0.889754343918514},
 'eval_precision': {'precision': 0.8617539440575972},
 'eval_recall': {'recall': 0.8640891083206289},
 'eval_roc_auc': {'roc_auc': 0.992628631348454},
 'eval_runtime': 82.9724,
 'eval_samples_per_second': 60.345,
 'eval_steps_per_second': 7.545,
 'epoch': 3.0}

Обучение завершено. Длилось около 45-50 минут.

Метрики хорошие: roc_auc близок к единице; precision, recall, accuracy более 0,85.

Сохраним модель локально

In [17]:
trainer.save_model("final_model_trained")

Saving model checkpoint to final_model_trained
Configuration saved in final_model_trained/config.json
Model weights saved in final_model_trained/pytorch_model.bin
tokenizer config file saved in final_model_trained/tokenizer_config.json
Special tokens file saved in final_model_trained/special_tokens_map.json


## **Итоги**

Мы обучили модель-трансформер DistilledBERT распознавать интенты с довольно высокой точностью

Используем первую строку из тестовой выборки для демонстрации инференса с помощью [`pipeline()`](https://huggingface.co/docs/transformers/v4.31.0/en/main_classes/pipelines#transformers.pipeline)

In [26]:
from transformers import pipeline

# для подавления вывода при загрузке модели
import logging
logging.disable(logging.INFO)

# загрузка предобученной модели из локальной директории
model = AutoModelForSequenceClassification.from_pretrained("./final_model_trained/")
tokenizer = AutoTokenizer.from_pretrained("./final_model_trained/")

# загрузка пайплайна и предскзание
pipe = pipeline(task="text-classification", model=model, tokenizer=tokenizer)
pipe.predict(df["test"][0]["text"])


[{'label': 'alarm_set', 'score': 0.996651828289032}]

Как можно заметить, модель выдает класс интента, который подходит по её мнению, и скор, который отображает, насколько интент предложения похож на интент из обучающей выборки

## Задание 2.

Что касается улучшения модели для учета out-of-scope запросов, есть две опции по обнаружению таких запросов:
- добавить дополнительную модель поверх - бинарный классификатор - которая распознает, out-of-scope ли запрос или нет
- вынести out-of-scope запросы в отдельный класс и использовать ту же модель

Для обоих методов, конечно, потребуются дополнительные данные, помеченные out-of-scope меткой, причем довольно много:
- неразборчивый текст/несвязный набор слов или букв
- запросы, не имеющие отношения к какой-либо теме (пустой диалог, например)
- запросы с неизвестным пока еще интентом


Насчет пользователей. Можно просто оповестить их сообщением "не могу удовлетворить ваш запрос" и прочие вариации. Однако есть идея поинтереснее: для улучшения пользовательского экспириенса, можно разбить out-of-scope класс на подклассы (по типу перечисления вариантов данных выше) и, исходя из подкласса, выдавать реакцию. Если смысл запроса не понятен вообще, то можно выдать просьбу о повторе запроса. Если смысл понятен, но не несет какого-либо интента, то можно продумать различные фразы с предложением какого-либо другого интента. Например: "не понимаю, о чем идет речь, но могу поставить будильник на завтра, чтобы вы не проспали". Если запрос неизвестен, то можно сделать так: извиниться и отправить специальную форму, где пользователь сам поставит метку своему запросу. Эта форма потом отправляется в data science отдел, где собираются дополнительные данные, и учитывается в последующем дообучении
